<a href="https://colab.research.google.com/github/ketanhdoshi/ml/blob/master/examples/Tensorboard_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Visualise with Tensorboard
This notebook has to be copied into other notebooks where you want to use Tensorboard

The next few examples are from this [article](https://www.easy-tensorflow.com/tf-tutorials/basics/introduction-to-tensorboard)

In [ ]:
import tensorflow as tf
print(tf.__version__)

1.12.0


In [ ]:
# Set the LOGDIR correctly to use Tensorboard
LOG_DIR = 'tbtry'

### Visualise a graph

In [ ]:
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

# create graph
a = tf.constant(2, name="const_a")
b = tf.constant(3, name="const_b")
c = tf.add(a, b, name="addop")

# launch the graph in a session
with tf.Session() as sess:
    # or creating the writer inside the session
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    print(sess.run(c))

5


### Visualise the Learning - Scalar and Histogram

In [ ]:
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

# ---------------- Create the Variables ---------------
# create the scalar variable
x_scalar = tf.get_variable('x_scalar', shape=[], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))

# create the matrix variable
x_matrix = tf.get_variable('x_matrix', shape=[30, 40], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))

# ---------------- Create the Summary ---------------
# create the scalar summary
scalar_summary = tf.summary.scalar(name='My_first_scalar_summary', tensor=x_scalar)

# A histogram summary for the non-scalar (i.e. 2D or matrix) tensor
histogram_summary = tf.summary.histogram('My_histogram_summary', x_matrix)

init = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # create the writer inside the session
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

    for step in range(100):
        # loop over several initializations of the variable
        sess.run(init)

        # evaluate the summaries
        summary1, summary2 = sess.run([scalar_summary, histogram_summary])

        # add the summary to the writer (i.e. to the event file)
        writer.add_summary(summary1, step)
        writer.add_summary(summary2, step)
    print('Done with writing the summaries')

Done with writing the summaries


### Visualise the Learning - Merged Scalar and Histogram

You need to run each summary and then use your writer to write each of them on the disk. In practice, you might use tens and hundreds of such summaries to track different parameters in your model. This makes running and writing the summaries extremly inefficient. The way around it is to merge all summaries in your graph and run them at once inside your session. This can be done using tf.summary.merge_all()

In [ ]:
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

# create the variables
x_scalar = tf.get_variable('x_scalar', shape=[], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))
x_matrix = tf.get_variable('x_matrix', shape=[30, 40], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))

# ____step 1:____ create the summaries
# A scalar summary for the scalar tensor
scalar_summary = tf.summary.scalar('My_scalar_summary', x_scalar)
# A histogram summary for the non-scalar (i.e. 2D or matrix) tensor
histogram_summary = tf.summary.histogram('My_histogram_summary', x_matrix)

# ____step 2:____ merge all summaries
merged = tf.summary.merge_all()

init = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # ____step 3:____ creating the writer inside the session
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    for step in range(100):
        # loop over several initializations of the variable
        sess.run(init)
        # ____step 4:____ evaluate the merged summaries
        summary = sess.run(merged)
        # ____step 5:____ add summary to the writer (i.e. to the event file) to write on the disc
        writer.add_summary(summary, step)
    print('Done writing the summaries')

Done writing the summaries


### Visualise the Learning - Images

In [ ]:
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

# create the variables
w_gs = tf.get_variable('W_Grayscale', shape=[30, 10], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))
w_c = tf.get_variable('W_Color', shape=[50, 30], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))

# ___step 0:___ reshape it to 4D-tensors
w_gs_reshaped = tf.reshape(w_gs, (3, 10, 10, 1))
w_c_reshaped = tf.reshape(w_c, (5, 10, 10, 3))

# ____step 1:____ create the summaries
gs_summary = tf.summary.image('Grayscale', w_gs_reshaped)
c_summary = tf.summary.image('Color', w_c_reshaped)

# create the op for initializing all variables
init = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # ____step 3:____ creating the writer inside the session
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    # initialize all variables
    sess.run(init)
    # ____step 4:____ evaluate the merged op to get the summaries
    summary1, summary2 = sess.run([gs_summary, c_summary])
    # ____step 5:____ add summary to the writer (i.e. to the event file) to write on the disc
    writer.add_summary(summary1, step)
    writer.add_summary(summary2, step)
    print('Done writing the summaries')

 

Done writing the summaries


### Histogram with [moving](https://www.tensorflow.org/guide/tensorboard_histograms) mean

Create two different histograms and a scalar. Also do two trial runs which are saved in different folders. This allows you to toggle between those runs in the Tensorboard visualisation

In [ ]:
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

k = tf.placeholder(tf.float32)

# Make a normal distribution, with a shifting mean
mean_moving_normal = tf.random_normal(shape=[1000], mean=(5*k), stddev=1)
# Record that distribution into a histogram summary
tf.summary.histogram("normal/moving_mean", mean_moving_normal)

# Make another normal distribution, with a shifting mean
mean_moving_normal_two = tf.random_normal(shape=[1000], mean=(12*k), stddev=4)
# Record that distribution into a histogram summary
tf.summary.histogram("normal/moving_mean_two", mean_moving_normal_two)

# create a scalar variable
moving_scalar = tf.get_variable('moving_scalar', shape=[], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))

moving_scalar = tf.add (moving_scalar, k)

# create the scalar summary
tf.summary.scalar(name='moving_scalar_summary', tensor=moving_scalar)

summaries = tf.summary.merge_all()

# Setup a session and summary writer
with tf.Session() as sess:
  for trial in range(2):
    gfolder = LOG_DIR + '/trial_' + str(trial)
    writer = tf.summary.FileWriter(gfolder, sess.graph)

    # Initialise the scalar to some value
    sess.run(tf.global_variables_initializer())

    # Setup a loop and write the summaries to disk
    N = 12
    for step in range(N):
      k_val = step/float(N)      
      summ = sess.run(summaries, feed_dict={k: k_val})
      writer.add_summary(summ, global_step=step)

folder is  tbtry/trial_0
scalar is  Tensor("Add:0", dtype=float32)
folder is  tbtry/trial_1
scalar is  Tensor("Add:0", dtype=float32)


### [Example](https://www.tensorflow.org/guide/tensorboard_histograms) of bi-modal distributions

In [ ]:
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

k = tf.placeholder(tf.float32)

# Make a normal distribution, with a shifting mean
mean_moving_normal = tf.random_normal(shape=[1000], mean=(5*k), stddev=1)
# Record that distribution into a histogram summary
tf.summary.histogram("normal/moving_mean", mean_moving_normal)

# Make a normal distribution with shrinking variance
variance_shrinking_normal = tf.random_normal(shape=[1000], mean=0, stddev=1-(k))
# Record that distribution too
tf.summary.histogram("normal/shrinking_variance", variance_shrinking_normal)

# Let's combine both of those distributions into one dataset
normal_combined = tf.concat([mean_moving_normal, variance_shrinking_normal], 0)
# We add another histogram summary to record the combined distribution
tf.summary.histogram("normal/bimodal", normal_combined)

summaries = tf.summary.merge_all()

# Setup a session and summary writer
sess = tf.Session()
writer = tf.summary.FileWriter(LOG_DIR)

# Setup a loop and write the summaries to disk
N = 12
for step in range(N):
  k_val = step/float(N)
  summ = sess.run(summaries, feed_dict={k: k_val})
  writer.add_summary(summ, global_step=step)

In [ ]:
!rm -r tbtry

### Set up Tensorboard

**To view Tensorboard output locally, use ngrok to tunnel traffic to localhost. First, download and unzip ngrok on the Colab server**

In [ ]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

**Get TensorBoard running in the background**

In [ ]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

**Launch ngrok background process**

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

**We get the public URL where we can access the colab TensorBoard web page. This will output a URL you can click on**

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://2536f14f.ngrok.io
